<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Copy_of_Antim_Bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# tf.config.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[0], 'GPU')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/drive/MyDrive/Major_Project/data.csv')

In [5]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


In [6]:
df.dropna(inplace=True)

In [7]:
df1=df[:1000]

In [8]:
df1.to_csv('data1.csv',index=False)

In [9]:
data=pd.read_csv('data1.csv')

In [10]:
data.shape

(1000, 6)

**Split the dataset into train and evaluate**

In [11]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2)

In [12]:
train_data.shape

(800, 6)

In [13]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')


In [14]:
import csv

# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [15]:
# import csv

# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [16]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00


In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [18]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


In [19]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)



/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [20]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


In [21]:
# Set up the trainer with the appropriate training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    save_total_limit=2,
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset
)



In [22]:
# Fine-tune the model on your dataset
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.412787
2,3.546100,3.385803
3,3.546100,3.382767


TrainOutput(global_step=939, training_loss=3.4312780540718184, metrics={'train_runtime': 582.0292, 'train_samples_per_second': 25.746, 'train_steps_per_second': 1.613, 'total_flos': 978865274880000.0, 'train_loss': 3.4312780540718184, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'eval_loss': 3.3827672004699707,
 'eval_runtime': 13.4455,
 'eval_samples_per_second': 91.555,
 'eval_steps_per_second': 11.454,
 'epoch': 3.0}

In [ ]:
trainer

In [23]:
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [38]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt').to('cuda'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


which is the best phone that i can buy?), but also for a device that is not a personal computer. But even without the cloud computing features of our own smartphones, this phone isn’t the  most flexible laptop computer around  because it is still a premium model  priced around $500 (  well below the likes of $580  you’ll have to spend to get it to run on a fully touchscreen or even a touch screen, which is great if you have dual screens but not worth buying because they are more expensive in terms of memory). The Pros:  Pros:  The Pros: Pros: Pros: Pros: They’re actually really good specs, which isn’t much compared to the $500+ price range  you’ve seen from Amazon and Asus, but the phone is still pretty damn good. There’s a huge amount of internal screen area to go around, which is really good considering that all of the apps,


In [29]:
trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
zip_ref.extractall()
zip_ref.close()